In [1]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup as BS
import os
import re

In [2]:
# Making the soup
url = 'https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/'
headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the pyhon-requests default user-agent
response = get(url, headers=headers)
soup = BS(response.content, 'html.parser')

In [3]:
def get_article_text(url):
    headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the pyhon-requests default user-agent
    response = get(url, headers=headers)
    soup = BS(response.content, 'html.parser')
    
    title = soup.title.text
    article = soup.main.article.text.replace('\xa0', ' ')
    
    return title, article

In [4]:
sites = [
    'https://codeup.com/codeups-data-science-career-accelerator-is-here/',
    'https://codeup.com/data-science-myths/',
    'https://codeup.com/data-science-vs-data-analytics-whats-the-difference/',
    'https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/',
    'https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

In [5]:
articles = [{'title' : get_article_text(site)[0],
             'body'  : get_article_text(site)[1]} for site in sites]

In [6]:
pd.DataFrame(articles)

,title,body
0,Codeup’s Data Science Career Accelerator is He...,Codeup’s Data Science Career Accelerator is He...
1,Data Science Myths - Codeup,"Data Science MythsPosted on October 31, 2018In..."
2,Data Science VS Data Analytics: What’s The Dif...,Data Science VS Data Analytics: What’s The Dif...
3,10 Tips to Crush It at the SA Tech Job Fair - ...,10 Tips to Crush It at the SA Tech Job FairPos...
4,Competitor Bootcamps Are Closing. Is the Model...,Competitor Bootcamps Are Closing. Is the Model...


In [7]:
sites = [
    'https://inshorts.com/en/read/business',
    'https://inshorts.com/en/read/sports',
    'https://inshorts.com/en/read/technology',
    'https://inshorts.com/en/read/entertainment']

In [8]:
url = 'https://inshorts.com/en/read/business'
headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the pyhon-requests default user-agent
response = get(url, headers=headers)
soup = BS(response.content, 'html.parser')

In [9]:
articles = soup.select('.news-card')

In [10]:
articles = [{'title'  : [article.find(itemprop="headline").text for article in articles],
             'body'   : [article.find(itemprop="articleBody").text for article in articles],
            'content' : url.split('/')[-1]}]

In [12]:
df = pd.DataFrame(articles)

In [13]:
df

,title,body,content
0,[Firm whose stock surged 1000% in 2020 starts ...,[US biotech company Novavax said it has starte...,business


In [16]:
def get_articles_from_topic(url):
    headers = {'User-Agent': 'Codeup Data Science'}
    response = get(url, headers=headers)
    soup = BS(response.content, 'html.parser')
    output = []
    articles = soup.select(".news-card")
    for article in articles: 
        title = article.select("[itemprop='headline']")[0].get_text()
        body = article.select("[itemprop='articleBody']")[0].get_text()
        author = article.select(".author")[0].get_text()
        published_date = article.select(".time")[0]["content"]
        category = response.url.split("/")[-1]
        article_data = {
            'title': title,
            'body': body,
            'category': category,
            'author': author,
            'published_date': published_date,
        }
        output.append(article_data)
    return output

In [24]:
def get_news_articles():
    urls = [
        "https://inshorts.com/en/read/business",
        "https://inshorts.com/en/read/sports",
        "https://inshorts.com/en/read/technology",
        "https://inshorts.com/en/read/entertainment"
    ]
    output = []
    for url in urls:
        output.extend(get_articles_from_topic(url))
    df = pd.DataFrame(output)
    df.to_csv('inshorts_news_articles.csv')
    return df

In [26]:
get_news_articles()

,title,body,category,author,published_date
0,Nokia shuts plant in Tamil Nadu after some emp...,Nokia on Tuesday said it suspended operations ...,business,Krishna Veera Vanamali,2020-05-27T04:10:52.000Z
1,Glenmark to start clinical trial to test poten...,Glenmark Pharmaceuticals on Tuesday said it wi...,business,Krishna Veera Vanamali,2020-05-26T11:22:33.000Z
2,Firm whose stock surged 1000% in 2020 starts h...,US biotech company Novavax said it has started...,business,Krishna Veera Vanamali,2020-05-26T10:17:41.000Z
3,India's economic growth seen at 1.2% in Q4 FY2...,India's economy is estimated to have grown at ...,business,Dharna,2020-05-26T12:41:03.000Z
4,"Twitter CEO donates $10M to project giving $1,...",Twitter's billionaire CEO Jack Dorsey has dona...,business,Pragya Swastik,2020-05-27T09:16:22.000Z
...,...,...,...,...,...
95,Mandana Karimi dismisses rumours of testing po...,Former Bigg Boss contestant and model-turned-a...,entertainment,Udit Gupta,2020-05-26T15:22:04.000Z
96,I overcame that: Madhuri on being called 'very...,When asked if she has heard anything nasty bei...,entertainment,Udit Gupta,2020-05-26T15:31:17.000Z
97,Never thought about casting myself for the ser...,On being asked if she ever thought about casti...,entertainment,Udit Gupta,2020-05-26T16:31:23.000Z
98,Akshay holds virtual meeting with team of 'Bel...,Akshay Kumar attended a virtual meeting with t...,entertainment,Udit Gupta,2020-05-26T17:07:15.000Z
